In [1]:
from qiskit import *
import numpy as np
import math

### Problema práctico 1 - Puertas clásicas con puertas cuánticas

Hemos visto durante este inicio del curso que las puertas lógicas cuánticas generalizan a las puertas clásicas, lo que quiere decir que con un circuito cuántico podemos representar cualquier circuito clásico. Para demostrar esto, crear, a partir de un circuito cuántico, las puertas $XOR$ y $NAND$. Las tablas de valores son las siguientes:

**XOR:**

| $A$ | $B$ | Out |
|:---:|:---:|:---:|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

**NAND:**

| $A$ | $B$ | Out |
|:---:|:---:|:---:|
|0|0|1|
|0|1|1|
|1|0|1|
|1|1|0|

Aquí teneis un ejemplo para la puerta $NOT$:

In [2]:
!pip install qiskit

In [5]:
from qiskit import *
def NOT(inp):

    qc = QuantumCircuit(1, 1) # this is where the quantum program goes
    
    # Codificaremos 0 -> |0> y 1 -> |1>. Por tanto, si nos entra un cero, aplicamos una puerta X 
    # para girar el qubit inicial.
    if inp==1:
        qc.x(0)
        
    # En este caso, el problema es sencillo, ya que la puerto NOT es equivalente a la puerta cuantica x:
    qc.x(0)
    
    # Medimos el estado del qubit
    qc.measure(0, 0)
    
    # Y corremos el circuito con el simulador
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc,backend)
    # Sacamos los valores de la ejecución del circuito
    output = next(iter(job.result().get_counts()))
    
    return output

print([NOT(0), NOT(1)])

['1', '0']


*Pista:* Podeís utilizar combinaciones de puertas y mediciones, al igual que *if*s clásicos. El único requisito es que el input pase por un circuito cuántico. 

In [ ]:
# Solución
from qiskit import *
a = 1
b = 0

qc = QuantumCircuit(3,1)

if (a==1) :
    qc.x(0)
if (b == 1) :
    qc.x(1)

qc.x(2)
qc.ccx(0,1,2)
backend = Aer.get_backend('qasm_simulator')
job = execute(qc,backend)
# Sacamos los valores de la ejecución del circuito
output = next(iter(job.result().get_counts()))



### Problema práctico 2 - Teleportación cuántica

Uno de los algoritmos más sencillos pero con más utilidad es el de teleportación cuántica. Es la base de lo que llamamos ahora comunicación cuántica. Permite enviar un estado, digamos $|\psi\rangle = \alpha|0\rangle+\beta|1\rangle$ de un punto A a un punto B. Existe un teorema en la física cuántica llamado teorema de no clonación, por el cual es imposible generar una copia exacta de un estado cuántico sin destruirlo. Esto tiene que ver con la decoherencia, ya que si quiero medir el estado $|\psi\rangle$ para poder generar una copia, dicha medición destruye directamente el estado. Para obetener los valores exactos de $\alpha$ y $\beta$, necesitaría medir el estado miles de veces para que las probabilidades de medir cada uno de los estado $|0\rangle$ y $|1\rangle$ convergiera a $\alpha$ y $\beta$.

Existe una manera de solucionar este problema, y es mediante el uso del entrelazamiento. Vamos a explicar el funcionamiento teórico del algoritmo y vuestra tarea sera la de recrear el algoritmo usando Qiskit.

**El algoritmo:**
El objetivo es mandar un qbit en el estado $|\psi\rangle$ desde el laboratorio de Alice hasta el de Bob
- **Paso 1:** Alice y Bob creando un par de qbit entrelazados, $q_A$ y $q_B$ y cada uno se guarda uno de ellos. Con una puerta h y otra CNA
- **Paso 2:** Alice aplica una puerta CNOT al qbit $q_A$, controlada por $|\psi\rangle$. Tenemos que aplicar cnot al primcipio
- **Paso 3:** Alice aplica una puerta Hadamard a $|\psi\rangle$ y mide los dos qubits $q_A$ y $|\psi\rangle$.
- **Paso 4:** Alice llama a Bob y le comunica el resultado de su medición. Dependiendo del resultado de la medición, Bob aplica una puerta y otra, de la manera siguiente:

| $|\psi\rangle$ | $q_A$ | Puerta de Bob |
|----|----|---------|
| $0$ | $0$ | No hacer nada |
| $0$ | $1$ | Aplicar la puerta $X$ |
| $1$ | $0$ | Aplicar la puerta $Z$ |
| $1$ | $1$ | Aplicar las puertas $ZX$|

Y ya está! Con esto Bob ha transformado su qbit $q_B$ al estado $|\psi\rangle$!

Para hacer nuestro caso un poco más educativo, vamos a crear un circuito mucho más grande del estrictamente necesario, con 5 qubits. Como hemos visto antes, solo necesitamos 3 para esta rutina. Iremos viendo para que nos sirven los otros dos. Nuestro objetivo es teletransportar $|\psi\rangle$, que tendremos en el qubit 0 circuito, al qubit de Bob, $q_B$, que en este caso será el qubit 2. El qubit 1 será el de Alice, $q_A$. Para crear $|\psi\rangle$, vamos a aplicar aleatoríamente una serie de puertas a $q_0$. Para guardarnos que estado hemos creado, vamos a aplicar esta puerta al qubit 3 también, de manera que $q_0$ y $q_3$ están en el mismo estado.

In [5]:
def secret_unitary(circuit, qubit):
    for _ in range(4):
        coin = np.random.rand()
        if coin < 0.5:
            circuit.h([q for q in qubit])
        else: 
            circuit.x([q for q in qubit])
            
q = QuantumRegister(5) # Vamos a considerar 2 qubit extras, a parte de los 
                       # tres necesarios. Veremos mas tarde su utilidad.
c = ClassicalRegister(1)
qc = QuantumCircuit(q, c)

secret_unitary(qc, [q[0], q[3]])
qc.barrier()

Ahora es vuestro turno! Crear el circuito en la siguiente celda. En nuestro caso, seguimos teniendo acceso al estado $|\psi\rangle$ y $q_A$, por lo que podemos sustituir las llamadas telefónicas por puertas controladas, teniendo en cuenta la tabla de más arriba.

In [6]:
# Solución

# Pas1 - Entrelazamiento
qc.h(q[1])
qc.cx(q[1],q[2])
qc.barrier()

# Paso2 
qc.cx(q(0),q[1])
qc.h(q[0])
qc.barrier()

#paso3
qc.cx(q[1],q[2])
qc.cz(q[0],q[2])
qc.barrier()

qc.draw(output = 'mpl')



TypeError: 'QuantumRegister' object is not callable

Una vez habeis creado el circuito, vamos a chequear que el estado actual de $q_B$ es exactamente el de $|\psi\rangle$. Existe multiples manera de hacerlo, pero una que es especilamente interesante cuando estamos trabajando con qubits es hacer un [*swap test*](https://en.wikipedia.org/wiki/Swap_test). El swap test actua de la manera siguiente: consideramos dos estados iniciales $|\psi\rangle$ y $|\phi\rangle$. Consideramos además un qubit auxiliar inicializado en $|0\rangle$. Si $|\langle \psi|\phi\rangle|^2=1$, es decir son el mismo estado, entonces la probabilidad de medir 0 en el ancilla qubit es 1. Si por el contrario no son iguales, la probabilidad de medir 0 es diferente de 1. El swap test tiene la siguiente forma:

<img src = '../figs/swap_test.png'>


Utilizando el swap test, comparar el estado de $|\phi\rangle$ que estaba en posesión de Alice con el estado actual de Bob, $q_B$. Como hemos aplicado una serie de puertas sobre $q_0$, este ya no esta en estado $|\phi\rangle$. Por suerte, $q_3$ si que lo está, ya que no lo hemos tocado desde el inicio. Utilizar como ancilla del swap test $q_4$. Ha funcionado la teleportación?

In [ ]:
# Solución

### Problema práctico 3 - Transformada de Fourier cuántica

En este problema trataremos de implementar las transformada de Fourier cuántica que hemos estudiado de manera teórica. Para ello, primero tenemos que definir la puerta $CROT$. Como ya hemos visto, se trata de una puerta que rota un qubit dependiendo de el valor del qubit de control. En `qiskit` encontramos la puerta de rotación $CU_1$, definida como
$$
CU_1(\theta) = 
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i\theta} \\
\end{pmatrix}
$$

En el caso de la puerta $CROT_k$, el ángulo de rotación es $\theta = 2\pi/2^k = \pi/2^{k−1}$. Un ejemplo de implementación de $CROT_2$ sería:

`quantum_circuit.cu(math.pi/2.0, quantum_register[0], quantum_register[1])`

Vuestra tare consiste en lo siguiente:
- Crear un circuito de 3 qubits.
- Aplicar la función input_state al circuito. Esta función crea un estado para el cual la QFT da como estado resultando el estado $1$ (en el caso de tres qubits, $|001\rangle$).
- Implementar la QFT a partir del circuito que habeís escrito.
- Utilizando el `qasm_simulator`, medir el resultado del circuito:

In [ ]:
def input_state(circ, q, n):
    """n-qubit input state for QFT that produces output 1."""
    for j in range(n):
        circ.h(q[j])
        circ.u1(-math.pi/float(2**(j)), q[j])

In [ ]:
# Solución

### Problema práctico 4 - Estimación de fase cuántica

El objetivo de este problema es extraer el valor $\theta$ dado una operación unitaria $U$ y un estado $|\psi\rangle$ de manera que $U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$. Para ello debeís utilizar el algoritmo de estimación de fase. En este ejercicio vamos a considerar $U=Z$ y $|\psi\rangle=|1\rangle$. Vamos a solucionar el problema con  $n=2$ qubits ancillas.
- Paso 1: dibujar el circuito, identificando las diferentes puertas a implementar y cual es el output esperado del circuito.
- Paso 2: calcular la forma de cada una de las puertas controladas a calcular.
- Paso 3: Implementar el circuito y apartir de este sacar el valor de $\theta$. Utilizar tanto el `qasm_simulator` como los chips cuánticos.
    - Primero, definir el circuito con 2 ancillas y 1 qubit extra, sobre el cual tendreís que aplicar una puerta $X$ para transformarlo al estado $|1\rangle$. Definir el circuito de tal manera que el qubit $|\psi\rangle$ este en la última posición.
    - Aplicar el algoritmo. Para la QFT inversa, podeís utilizar la siguiente función `qft_inversa` que encontrareís más abajo.
    - Medir el estado de las ancillas, pasar el valor del qubit de binario a decimal y sacar el valor de $\theta$.

In [ ]:
def qft_inversa(circuito, q, n):
    '''Inputs:
        - circuito: QuantumCircuit
        - q: QuantumRegister
        - n: Numero de ancillas '''
    for j in range(n):
        k = (n-1) - j
        for m in range(k):
            circuito.cu1(-math.pi/float(2**(k-m)), q[k], q[m])
        circuito.h(q[k])

In [ ]:
# Solución